[View in Colaboratory](https://colab.research.google.com/github/seonhonam/RC/blob/master/ESN.ipynb)

In [0]:
import tensorflow as tf
import numpy as np
import networkx as nx

In [0]:
n_inputs = 10
n_reservoir = n_inputs*100
n_readout_hidden1 = 400
n_readout_hidden2 = 400
n_readout_hidden3 = 200
n_readout_outputs = 10
n_steps = 10

In [0]:
X = tf.placeholder(tf.float32, [None, n_inputs], name = "X")
y = tf.placeholder(tf.float32, [None, n_readout_outputs], name = "y")

In [0]:
#reservoir layer를 만들어 준다
def reservoir_layer(X,n_reservoir,internal_network, before_reservoir_state,
                   name):
  with tf.name_scope("name"):
    n_inputs = int(X.get_shape()[1])
    stddev = 2/np.sqrt(n_inputs + n_reservoir)
    W_in = tf.Variable(tf.truncated_normal((n_inputs, n_reservoir),
                                           stddev = stddev))
    W = tf.Variable(internal_network)
    if before_reservoir_state == 0:
      Z = tf.matmul(X, W_in)
    else:
      Z = tf.matmul(X, W_in)+tf.matmul(before_reservoir_state, W)
    return Z

In [0]:
# network에 들어갈 adjacency matrix를 만든다
#ER Random network의 adjacency matirx
def ERN(n_node,p):
  g = nx.erdos_renyi_graph(n_node,p,seed=2,directed=True)
  adm = np.array(nx.adj_matrix(g).todense())
  return adm
inter_net = ERN(n_reservoir,0.4)

In [0]:
#네트워크 모델을 만든다
with tf.name_scope("ESN"):
  before_reservoir = 0
  reservoir= reservoir_layer(X, n_reservoir, inter_net, before_reservoir, 
                             name="reservoir")
  before_reservoir = reservoir
  reservoir_stop = tf.stop_gradient(reservoir)
  hidden1 = tf.layers.dense(reservoir, n_readout_hidden1, name = "hidden1", 
                           activation = tf.nn.elu)
  hidden2 = tf.layers.dense(hidden1, n_readout_hidden2, name = "hidden2",
                           activation = tf.nn.elu)
  hidden3 = tf.layers.dense(hidden2, n_readout_hidden3, name = "hidden3",
                           activation = tf.nn.elu)
  logits = tf.layers.dense(hidden3, n_readout_outputs)